<h3>0. Exploratory analysis</h3>

In [1]:
# tools
import pickle

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [29]:
df_training.head()

,answer_paragraph,docid,question,text,NE_question,NE_text,NE_paragraph,answer_type,keywords_question,question_type,POS_questions,answer_found,corrected_answer,NE_corrected_answer,NE_answer_found,label
0,23,0,A kilogram could be definined as having a Plan...,6966662606895999999♠6.62606896×10−34 j⋅s,[],"[(6966662606895999999 ♠ 6.62606896, NUMBER), (...","[[(general, TITLE), (2011, DATE)], [], [(one, ...",NUMBER,"[kilogram, definined, planck, constant, value]",what,"[NN, VBN, NNP, NN, NN]","Possible new definitions include ""the mass of ...",6966662606895999999♠6.62606896×10−34 J⋅s,"[(6966662606895999999 ♠ 6.62606896, NUMBER), (...","[(7050135639273999999 ♠ 135639274 ×, NUMBER), ...",NUMBER
1,22,0,What is the shape of the object that establish...,cylinder,[],[],"[[], [], [(1889, DATE), (paris, CITY)], [(1889...",None,"[shape, object, establish, base, unit, kilogram]",what,"[NN, NN, VBZ, JJ, NN, NN]",The most urgent unit on the list for redefinit...,cylinder,[],"[(1889, DATE), (paris, CITY)]",OTHER
2,12,0,What example is given as another paired relati...,time vs. energy,[],[],"[[], [], [(one, NUMBER)], [(fourier, LOCATION)]]",None,"[example, give, pair, relationship, relate, st...",what,"[NN, VBN, JJ, NN, VBN, JJ, NN]",One example is time vs. energy.,time vs. energy,[],"[(one, NUMBER)]",OTHER
3,1,0,What does the Planck Constant refer to?,quantum of action,[],[],"[[], [(planck, PERSON)], [(now, DATE)], [], []]",None,"[doe, planck, constant, refer]",what,"[VBZ, NNP, NNP, NN]","Instead, it must be some multiple of a very sm...",quantum of action,[],"[(now, DATE)]",OTHER
4,10,0,When was the first quantized model of the atom...,1913,"[(first, ORDINAL), (model, TITLE)]","[(1913, DATE)]","[[(niels bohr, PERSON), (first, ORDINAL), (mod...",DATE,"[wa, first, quantize, model, atom, introduce]",when,"[VBD, JJ, JJ, NN, NN, VBD]",Niels Bohr introduced the first quantized mode...,1913,"[(1913, DATE)]","[(niels bohr, PERSON), (first, ORDINAL), (mode...",TIME


In [100]:
import json
import pandas as pd

df_devel=pd.read_pickle('project_files/df_devel.pkl')
df_docs=pd.read_json('project_files/documents.json')
df_testing=pd.read_pickle('project_files/df_testing.pkl')
df_training=pd.read_pickle('project_files/df_training.pkl')
question_learning_dataset = df_training[df_training.answer_type.notnull()]
question_devel_dataset = df_devel[df_devel.answer_type.notnull()]
NER_corpus=load_obj('ner_corpus')
model = load_obj('random_forest')

In [3]:
print(len(question_learning_dataset),len(question_devel_dataset))

29174 1969


In [10]:
from nltk.tokenize.punkt import PunktSentenceTokenizer,PunktTrainer

tokenizer = load_obj('punk_tokenizer')
tokenizer._params.abbrev_types.add('ii')
tokenizer._params.abbrev_types.add('dr')

questionwords = set(["who", "what", "where", "when", "why", "how", "whose", "which", "whom","whats","what's","whos"])
passiveQuestions = set(["can", "could", "would", 
                   "was", "were","am","is", "are", "will","shall",
                   "did","do","does",
                   "had", "have","has",
                   "as","that","in",
                   "give an example","name"])


1. Find Keywords
2. Answer types - Using answer type taxonomy
3. Query formulation -> Keywords
4. Go to each document and check the frequency distribution of words and pick the document if one of the query words are present in document. Create a rank with that score
5. Find the paragraphs -> Discard irrelevant paragraphs. Use NE,Keywords, longest exact keywords. Put same weight for now and calculate the score of paragraphs. Rank each of the paragraphs in the document. We have to use the original answer and match the answer type
6. Find candidate answers -> Use supervised ML method
7. Merge candidate answers -> Use NER
8. Pick the best answer -> Logistic regression

<h3>1. Question processing</h3>

Configuring Stanford CoreNLP . Link -> https://blog.manash.me/configuring-stanford-parser-and-stanford-ner-tagger-with-nltk-in-python-on-windows-f685483c374a

In [11]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tag.stanford import CoreNLPNERTagger
from itertools import groupby

stopwords = set(nltk.corpus.stopwords.words('english')) 


def get_Name_Entity_NLTK(data):
    results=[]
    for sentence in data:
        ne_chunked_sents = ne_chunk(pos_tag(word_tokenize(sentence)))
        result = []

        for tagged_tree in ne_chunked_sents:

            if hasattr(tagged_tree, 'label'):
                entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #
                entity_type = tagged_tree.label() # get NE category
                result.append((entity_name, entity_type))
        results.append(result)

    return results

def get_Name_Entity_Sentence(sentence):
    st = CoreNLPNERTagger(url='http://localhost:9000')
    tokenized_text = nltk.word_tokenize(sentence)
    classified_text = st.tag(tokenized_text)
    result = []
    
    for tag, chunk in groupby(classified_text, lambda x:x[1]):
       if tag != "O":
            word = " ".join(w for w, t in chunk)
            result.append((word.lower(), tag))
    
    return result


def addNameEntity(df,feature,func):
    if 'NE'+"_"+feature in df:
        df = df.drop('NE'+"_"+feature, axis=1)
    df["NE"+"_"+feature] = func(df[feature])
    
    return df

In [206]:
def get_question_type(question):
    found  = False
    result = 'other'
    question_tokens = nltk.word_tokenize(question)
    for token in question_tokens:
        if token in questionwords:
            found = True
            result = token
    if not found:
        for token in question_tokens:
            if token in passiveQuestions:
                found = True
                result = token

    return result

Get Keywords

In [12]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

POS = set(["NN","NNS","NNP","NNPS","CD","JJ","VB","VBD","VBG","VBN","VBP","VBZ"]) 

stopwords = set(nltk.corpus.stopwords.words('english')) 


def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,wn.NOUN)
    if (lemma == word):
        lemma = lemmatizer.lemmatize(word,wn.VERB)
        
    return lemma

def get_keyword(data):
    result = []
    sentence=data
    tokenized_text = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokenized_text)
    for text,pos in tagged:
        text = lemmatize(text.lower())
        if text not in stopwords:
            if pos in POS:
                result.append(text)
                
    return result

def get_keyword_paragraph(data):
    results=[]
    tokenized_sentence = tokenizer.tokenize(data)
    for sentence in tokenized_sentence:
        result = get_keyword(sentence)
        results.append(result)
        
    return results

def get_keyword_all(data):
    results=[]
    for sentence in data:
        result = get_keyword(sentence)
        results.append(result)
        
    return results

def add_keywords(df,feature):
    if 'keywords'+"_"+feature in df:
        df = df.drop('keywords'+"_"+feature, axis=1)
    df['keywords'+"_"+feature]=get_keyword_all(df[feature])
    return df

def get_number_of_common_kewyords(question_keywords,answer_sentence_keywords):
    sum_keywords=0
    for qkey in question_keywords:
        if qkey in answer_sentence_keywords:
            sum_keywords+=1
    
    return sum_keywords

<h4>Train a classifier</h4>

In [5]:
# BOW extraction for passages and questions
def get_passages_bow(passages):
    passage_bow={}
    for passage in passages:
        for token in nltk.word_tokenize(passage):
            if token not in stopwords: 
                word=lemmatize(token.lower())
                passage_bow[word] = passage_bow.get(word, 0) +  1
    
    return passage_bow

def get_sentences_bow(sentences):
    sentence_bow={}
    
    for sentence in sentences:
        for token in nltk.word_tokenize(sentence):
            if token not in stopwords:
                word=lemmatize(token.lower())
                sentence_bow[word] = sentence_bow.get(word, 0) +  1
    
    return sentence_bow

def get_question_bow(question):
    question_bow={}
    question_bow[get_question_type(question)]=1
    for token in nltk.word_tokenize(question):
        if token not in stopwords: 
            word=lemmatize(token.lower())
            question_bow[word] = question_bow.get(word, 0) +  1
                
    return question_bow

def get_training_question_bow(question,keywords,qt):
    question_bow={}
    question_bow[qt]=1
    for token in nltk.word_tokenize(question):
        if token not in stopwords: 
            word=lemmatize(token.lower())
            if word in keywords:
                question_bow[word] = question_bow.get(word, 0) +  1
                
    return question_bow

In [220]:
def get_feature_questions(questions, keywords,qt):
    qs = []
    for i,question in enumerate(questions):
        q_bow = get_training_question_bow(question,keywords,qt[i])
        qs.append(q_bow)
        
    return qs
def get_feature_question(question, keywords,qt):
#     print(question,keywords,qt)
    q_bow = get_training_question_bow(question,keywords,qt)
      
    return q_bow

In [8]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def check_results(predictions, classifications):
    print("Accuracy:")
    print(accuracy_score(classifications,predictions))
    print(classification_report(classifications,predictions))

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer


    
# get the most common words from answer sentences (we can twek this for paragraph)
answer_sentences_bow=get_sentences_bow(question_learning_dataset[question_learning_dataset['answer_found'].notnull()]['answer_found'])
answer_keywords = set([word for word, count in answer_sentences_bow.items()])

#qs_training=get_feature_questions(questions,answer_keywords)
qs_training=get_feature_questions(list(question_learning_dataset.question),answer_keywords,list(question_learning_dataset.question_type))
qs_dev=get_feature_questions(list(question_devel_dataset.question),answer_keywords,list(question_devel_dataset.question_type))


In [14]:
from sklearn.ensemble import RandomForestClassifier

if (len(qs_training)>0 and len(list(question_learning_dataset.question_type))>0):
    # fit vectorizer
    vectorizer = DictVectorizer()
    
    X_train_dtm = vectorizer.fit_transform(qs_training)
    X_dev_dtm = vectorizer.transform(qs_dev)
    

    model=RandomForestClassifier(n_estimators = 300, max_depth = 60, criterion = 'entropy')
    
    # tag the answers
    # fit a logistic regression model to the data 
    # build classifier
    #model = MultinomialNB(2, False, None)

    # train the model using X_train_dtm 
    model.fit(X_train_dtm, list(question_learning_dataset.label))
    
    y_predicted_class = model.predict(X_dev_dtm)
    check_results(y_predicted_class,list(question_devel_dataset.label))

Accuracy:
0.7145759268664297
              precision    recall  f1-score   support

    LOCATION       0.70      0.53      0.60       226
        MISC       0.00      0.00      0.00        51
        NAME       0.79      0.25      0.38       184
      NUMBER       0.90      0.78      0.83       466
ORGANIZATION       0.75      0.08      0.15        72
      PERSON       0.56      0.91      0.70       548
        TIME       0.85      0.88      0.87       422

 avg / total       0.73      0.71      0.69      1969



C:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [217]:
model.predict(X_dev_dtm[0])

array(['TIME'], dtype='<U12')

In [17]:
save_obj(model,'random_forest1')
save_obj(vectorizer,'Vectorizer')
save_obj(qs_training,'qs_training')
save_obj(qs_dev,'qs_dev')

<h3>2. Candidate answering generation</h3>

<h4> Get a score for the passage to filter the most relevant passages</h4>


First, we will set up useful functions to extract term frequencies to build the vector space model

In [23]:
# create a document-term matrix
docs=get_document_term_passsages(df_docs)
#docs

In [24]:
# create a vector space model we need to define a score function
# first I will use tf-idf
doc_term_freqs = {}
for docid,dic_passages in docs.items():
    passage_dic = {}
    for passage_id, terms in dic_passages.items():
        term_freqs = extract_term_freqs(terms)
        passage_dic[passage_id] = term_freqs
    doc_term_freqs[docid] = passage_dic

doc_freqs = compute_doc_freqs(doc_term_freqs)


<b>Improvement:</b> Use BM25

Create an inverted index for query processing. Inverted index will not change from query to query. Here we can improve how the weight is defined for the posting list tuple for each term (docid,weight)

In [18]:
vsm_inverted_index_all=load_obj('vsm_inverted_index_corpus')

Query the VSM creating a score for each document (passage) and returning the top k

In [19]:
from collections import Counter
# get a list of paragraphs ordered by relevance on the question
def query_vsm(query, index):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator

## end copied code

<h3>3. Candidate answering scoring</h3>

In [21]:
import re
def correct_answer_space(predicted,predicted_answer_sentence):
    
    tokens=nltk.word_tokenize(predicted)
    pattern='.*('
    for token in  tokens:
        pattern=pattern+token+'\s*'
    pattern=pattern+').*'

    reg=re.compile(pattern,re.IGNORECASE)
    if len(re.findall(reg,predicted_answer_sentence)):
        predicted=re.findall(reg,predicted_answer_sentence)[0].strip()
    
    return predicted

def correct_answer_pattern(predicted):
    corrected=predicted
    
    # symbol % based pattern
    pattern_percentaje='(.*[0-9])(%.*)'
    reg=re.compile(pattern_percentaje,re.IGNORECASE)
    result=re.findall(reg,predicted)
    if len(result)>0:
        groups=result[0]
        corrected=groups[0]+' '+groups[1]
    
    
    # date pattern
    #pattern_date='(Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s+(\d{1,2})(,.*)'
    pattern_date='(January|February|March|April|May|June|July|August|September|October|November|December)\s+([0-9]{1,2})(,.*)([0-9]{4})'


    reg=re.compile(pattern_date,re.IGNORECASE)
    result=re.findall(reg,predicted)

    if len(result)>0:
        groups=result[0]   
        corrected=(groups[0]+' '+groups[1]+' '+groups[2]+groups[3])


    return corrected
    
    
def isAnswerInSentence(answer,answer_sentence):
    #print('Eval:',answer)
    #print('In:',answer_sentence)
    tokens=nltk.word_tokenize(answer)
    pattern='.*('
    for token in  tokens:
        pattern=pattern+token+'\s*'
    pattern=pattern+').*'

    
    reg=re.compile(pattern,re.IGNORECASE)
    if len(re.findall(reg,answer_sentence))>0:
        return True
    
    return False




In [ ]:
label={'NUMBER':'NUMBER','DATE':'TIME','PERSON':'PERSON','ORGANIZATION':'ORGANIZATION',
      'MISC':'MISC','MONEY':'NUMBER','COUNTRY':'LOCATION',
      'PERCENT':'NUMBER','TITLE':'PERSON','STATE_OR_PROVINCE':'LOCATION',
      'CAUSE_OF_DEATH':'NAME','DURATION':'TIME','CRIMINAL_CHARGE':'NAME',
       'CITY':'LOCATION','RELIGION':'NAME','SET':'TIME','NATIONALITY':'NAME',
       'IDEOLOGY':'NAME','ORDINAL':'NUMBER','TIME':'TIME','URL':'ORGANIZATION',None:'OTHER','LOCATION':'LOCATION'}

In [275]:
df_devel1.head()

,answer_paragraph,docid,question,text,question_type,NE_question,NE_text,keywords_question,answer_type,POS_questions,answer_found,corrected_answer,NE_corrected_answer,corrected_answer_type,label,NE_answer_found
0,5,380,On what date did the companies that became the...,"june 16 , 1911",what,[],"[(june 16 , 1911, DATE)]","[date, company, become, computing-tabulating-r...",DATE,"[NN, NNS, VBD, NNP, NNP, VB, VBN]","On June 16, 1911, their four companies were co...","June 16, 1911","[(june 16 , 1911, DATE)]",DATE,TIME,"[(june 16 , 1911, DATE), (four, NUMBER), (new ..."
1,22,380,What percentage of its desktop PCs does IBM pl...,5 %,what,"[(ibm, ORGANIZATION)]","[(5 %, PERCENT)]","[percentage, desktop, pc, doe, ibm, plan, inst...",PERCENT,"[NN, JJ, NN, VBZ, NNP, NN, VB, NNP, NNP]",IBM plans to install Open Client on 5% of its ...,5%,"[(5 %, PERCENT)]",PERCENT,NUMBER,"[(ibm, ORGANIZATION), (5 %, PERCENT)]"
2,16,380,What year did IBM hire its first black salesman?,1946,what,"[(year, DURATION), (ibm, ORGANIZATION), (first...","[(1946, DATE)]","[year, ibm, hire, first, black, salesman]",DATE,"[NN, NNP, VB, JJ, JJ, NN]","In 1946, the company hired its first black sal...",1946,"[(1946, DATE)]",DATE,TIME,"[(1946, DATE), (first, ORDINAL), (salesman, TI..."
3,4,380,"IBM made an acquisition in 2009, name it.",spss,in,"[(ibm, ORGANIZATION), (2009, DATE)]",[],"[ibm, make, acquisition, 2009, name]",ORGANIZATION,"[NNP, VBD, NN, CD, NN]",IBM acquired Kenexa (2012) and SPSS (2009) and...,SPSS,"[(spss, ORGANIZATION)]",ORGANIZATION,ORGANIZATION,"[(ibm, ORGANIZATION), (kenexa, PERSON), (2012,..."
4,2,380,This IBM invention is known by the acronym UPC...,universal product code,what,"[(ibm, ORGANIZATION), (upc, ORGANIZATION)]",[],"[ibm, invention, know, acronym, upc, full, name]",None,"[NNP, NN, VBN, NN, NNP, JJ, NN]",Notable company inventions or developments inc...,Universal Product Code,[],None,OTHER,"[(teller, TITLE), (upc, ORGANIZATION), (fortra..."


In [261]:
df_result_devel=pd.DataFrame(columns=['id','question','paragraph','retrieved paras','predicted_paragraph',
                                      'paragraph_found','sentence','predicted_sentence','predicted_sentence_id',
                                      'sentence_found','answer','predicted_answer','NE_question','label'])
df_devel1=df_devel.iloc[0:100]
count = 0
for index, row in df_devel1.iterrows():
    question=row['question']
    docid=row['docid']
    ida=index
    question_keywords=get_keyword(question)
    results = query_vsm(question_keywords, vsm_inverted_index_all[docid])
    documents_ranked=results.most_common(10) 
    best_dic = set()
    cc = 0
    found = False
    possible_par=[par[0] for par in documents_ranked]
    best_paragraph_id = -1
    par_retrieved = False
    for doc in documents_ranked:
        if(cc>1):
            break
        cc = cc + 1
        best_dic.add(doc[0])
    if(row['answer_paragraph'] in best_dic):
        count = count + 1
        par_retrieved = True
        best_paragraph_id = row['answer_paragraph']
    df_result_devel.loc[ida] = [docid,question,row['answer_paragraph'],
                                possible_par,best_paragraph_id,
                                par_retrieved,row['answer_found'],None,None,False,row['text'],None,
                                row['NE_question'],row['label']]

print(count*1.0/len(df_devel1))

0.93


In [262]:
df_result_devel = df_result_devel[df_result_devel['paragraph_found']==True]
print(len(df_result_devel))

93


In [263]:
df_best_sentences=pd.DataFrame(columns=['doc_id','para_id','original','sentence_id','sentence_text','score','sentence'])

for index,row in df_result_devel.iterrows():
    NE_question = row['NE_question']
    paragraph = df_docs.iloc[row['id']]['text'][row['predicted_paragraph']]
    paragraph_id = row['predicted_paragraph']
    question = row['question']
    answer = row['sentence']
    docid = row['id']
    candidate_answers = tokenizer.tokenize(paragraph)
    question_keywords=get_keyword(question)
    sentences_check = Counter()
    for sentence_index,sentence in enumerate(candidate_answers):
        NER_sentence=NER_corpus[docid][paragraph_id][sentence_index]
        answer_sentence_keywords = get_keyword(sentence)
        common_keywords=get_number_of_common_kewyords(question_keywords,answer_sentence_keywords)
        common_entities = get_number_of_common_entities(NER_sentence,NE_question)
        
        # longest exact sequence of keywords
        longest_exact_sequence=0
        for i in range(len(question_keywords)):
            if i < len(answer_sentence_keywords):
                if question_keywords[i] in answer_sentence_keywords[i]:
                    longest_exact_sequence+=1
#                     print(question_keywords[i],answer_sentence_keywords[i])
#         print("Longest->",longest_exact_sequence)
        
        # proximity
        proximity=0
        question_keywords_span=question_keywords.copy()
        index_qk=0
        while len(question_keywords_span)>0 and index_qk<len(question_keywords_span):
            proximity+=1
            if question_keywords_span[index_qk] in answer_sentence_keywords:
                question_keywords_span.pop(index_qk)
            index_qk+=1
#         print("proximity->",proximity)
        
        # n-gram overlap
        bigrams_question =  nltk.bigrams([lemmatize(token) for token in nltk.word_tokenize(question)])
        ngram_overlap=0

        for bigram_question in bigrams_question:
            bigrams_sentence = nltk.bigrams([lemmatize(token) for token in nltk.word_tokenize(sentence)])
            for bigram_sentence in bigrams_sentence:
                if bigram_question == bigram_sentence:
#                     print(bigram_question,'sentence',bigram_sentence)
                    ngram_overlap+=1
#         print("n-gram->",ngram_overlap)
        sentence_score = common_keywords + common_entities + longest_exact_sequence + proximity + ngram_overlap
        df_best_sentences.loc[len(df_best_sentences)]=[row['id'],
                                                       paragraph_id,row['paragraph'],sentence_index,
                                                       sentence,sentence_score,row['sentence']]
        sentences_check[sentence_index] = sentence_score
    
    best = sentences_check.most_common()
    print(index)
    df_result_devel.at[index, 'predicted_sentence_id'] = best[0][0]
    df_result_devel.at[index, 'predicted_sentence'] = candidate_answers[best[0][0]]
print("Done")

0
1
2
4
5
6
8
9
10
11
12
13
14
15
16
17
18
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Done


In [264]:
count  = 0
for index,row in df_result_devel.iterrows():
    sentence = row['sentence']
    predicted_sentence = row['predicted_sentence']
    if(sentence==predicted_sentence):
        count = count + 1
        df_result_devel.at[index, 'sentence_found'] = True
print(count*1.0/len(df_result_devel))

0.7741935483870968


In [113]:
def get_number_of_common_entities(ans,question):
    question_NE = set(x[1] for x in question)
    answer_NE = set(x[1] for x in ans)
    sum_keywords=0
    for qkey in question_NE:
        if qkey in answer_NE:
            sum_keywords+=1
#     print(ans,question)
    return sum_keywords

In [266]:
df_result_devel1 = df_result_devel[df_result_devel['sentence_found']==True]
print(len(df_result_devel1))

72


In [267]:
count = 0

df_result_devel1['predicted_label'] = None

for index,row in df_result_devel1.iterrows():
    sentence = row['predicted_sentence']
    question = row['question']
    question_NER = row['NE_question']
    sentence_id = row['predicted_sentence_id']
    paragraph_id = row['predicted_paragraph']
    docid = row['id']
    answer = row['answer']
    sentence_NER = NER_corpus[docid][paragraph_id][sentence_id]
    sentence_keywords = get_keyword(sentence)
    question_keywords=get_keyword(question)
    predicted_label = model.predict(vectorizer.transform(
        get_feature_question(question,answer_keywords,get_question_type(question))))[0]
    
    df_result_devel1.at[index, 'predicted_label'] = predicted_label
    
    answer_type = None
    predicted_answer = None
    for NE in sentence_NER:
        if (label[NE[1]]==predicted_label):
            answer_type = NE[1]
            predicted_answer = NE[0]
            break
#     print(index,answer,predicted_label,answer_type,predicted_answer)
    if(answer_type):
        df_result_devel1.at[index, 'predicted_answer'] = predicted_answer
        count = count + 1
#     print(index)
print(count)
#     else:
#         print(sentence,""predicted_sentence)
#         print(index,answer,predicted_label,answer_type)
#         break
#     print(index,question,paragraph_id,sentence_id,
#           sentence,sentence_NER,sentence_keywords,question_keywords,answer,predicted_label)

C:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


45


In [270]:
len(df_result_devel1)

72

In [278]:
count = 0
for index,row in df_result_devel1.iterrows():
    ans = row['label']
    pred = row['predicted_label']
    if(ans == pred):
        count  = count + 1
print(count)

32


In [280]:
import difflib

count = 0
count1 = 0
count3 = 0
for index,row in df_result_devel1.iterrows():
    ans = row['answer']
    pred = row['predicted_answer']
    if(row['label']==row['predicted_label']):
        if(ans == pred):
            count = count + 1
        else:
            if(pred):
                seq=difflib.SequenceMatcher(None, ans,pred)
                d=seq.ratio()*100
                if (d>20.00):
    #                 print(index,row['question'])
                    print(row['label'],"~~",row['predicted_label'],'~~',ans,"~~",pred)
                    count1 = count1 + 1
                else:
    #                 print(index,row['question'])
                    print(row['label'],"~~",row['predicted_label'],'~~',ans,"~~",pred)
                    count3 = count3 + 1
            else:
                print("strange",pred,row['answer'],"~~",row["predicted_answer"])
print(count,count1,count3)

NUMBER ~~ NUMBER ~~ second largest ~~ second
NUMBER ~~ NUMBER ~~ 4,100 gallons ~~ six
TIME ~~ TIME ~~ '07 ~~ november 2010
PERSON ~~ PERSON ~~ blue gene ~~ gene
PERSON ~~ PERSON ~~ albert l. williams ~~ thomas j. watson
NUMBER ~~ NUMBER ~~ 34 ~~ first
TIME ~~ TIME ~~ 2013 ~~ 2004
LOCATION ~~ LOCATION ~~ the u.s. south ~~ u.s.
strange None charles ranlett flint ~~ None
NUMBER ~~ NUMBER ~~ 78,000 gallons ~~ 78,000
NUMBER ~~ NUMBER ~~ five nobel prizes ~~ five
NUMBER ~~ NUMBER ~~ fourth largest ~~ second
NUMBER ~~ NUMBER ~~ 26 million ~~ first
NUMBER ~~ NUMBER ~~ 5.5 ~~ 5.5 million
NUMBER ~~ NUMBER ~~ 53 ~~ 400
LOCATION ~~ LOCATION ~~ tennessee ~~ oak ridge
16 8 7
